# Hybrid Forgery Training + Evaluation
This notebook lets you tune every training hyperparameter, toggle ablation flags, and launch `run_training` with tqdm progress bars. After training, reuse the same configuration to load checkpoints, compute metrics (Dice/IoU/precision/recall/F1 + confusion matrix), and visualize 10 qualitative test samples with image / ground-truth / prediction / overlay columns.

In [ ]:
import os
import subprocess
import shutil
from pathlib import Path

REPO_URL = "https://github.com/DeogenesMaranan/LIFD"
COLAB_REPO_DIR = Path("/content/LIFD")


def running_in_colab() -> bool:
    try:
        import google.colab
        return True
    except ImportError:
        return False


USE_COLAB = running_in_colab()

if USE_COLAB:
    if COLAB_REPO_DIR.exists():
        print(f"Folder {COLAB_REPO_DIR} already exists. Deleting it...")
        shutil.rmtree(COLAB_REPO_DIR)

    print(f"Cloning {REPO_URL} -> {COLAB_REPO_DIR}")
    subprocess.run(["git", "clone", REPO_URL, str(COLAB_REPO_DIR)], check=True)

    os.chdir(COLAB_REPO_DIR)
    print(f"Working directory set to {Path.cwd()}")
else:
    print("Colab environment not detected; using current local working directory.")

In [ ]:
DRIVE_BASE_DIR = "/content/drive/MyDrive/LIFD"
if USE_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive')
    except ModuleNotFoundError as exc:
        raise RuntimeError("google.colab is not available. Set USE_COLAB=False to bypass Drive mounting.") from exc
else:
    DRIVE_BASE_DIR = "."

In [ ]:
from copy import deepcopy
from dataclasses import asdict
from pathlib import Path
from pprint import pprint
import math
import os
import shutil
from zipfile import ZipFile

import matplotlib.pyplot as plt
import numpy as np
import torch

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

try:
    import pandas as pd
except ImportError:
    pd = None

try:
    import requests
except ImportError:
    requests = None

try:
    from huggingface_hub import hf_hub_download, snapshot_download
except ImportError:
    hf_hub_download = None
    snapshot_download = None

try:
    from google.colab import userdata
except ImportError:
    userdata = None

torch.set_float32_matmul_precision("high")
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True

CPU_COUNT = os.cpu_count() or 4
GPU_AVAILABLE = torch.cuda.is_available()
if GPU_AVAILABLE:
    try:
        _gpu_props = torch.cuda.get_device_properties(0)
        GPU_TOTAL_MEM_GB = _gpu_props.total_memory / (1024 ** 3)
    except Exception:
        GPU_TOTAL_MEM_GB = None
else:
    GPU_TOTAL_MEM_GB = None

env_perf_mode = os.environ.get("LIFD_PERF_MODE", "").strip().lower()
if env_perf_mode in {"fast", "balanced", "throughput"}:
    PERFORMANCE_MODE = env_perf_mode
else:
    if GPU_AVAILABLE and CPU_COUNT >= 8:
        PERFORMANCE_MODE = "throughput"
    elif CPU_COUNT >= 6:
        PERFORMANCE_MODE = "balanced"
    else:
        PERFORMANCE_MODE = "fast"

if PERFORMANCE_MODE == "throughput":
    DATA_WORKERS = max(4, min(8, CPU_COUNT - 1))
    TRAIN_BATCH_SIZE = 32 if GPU_AVAILABLE else 8
    GRAD_ACCUM_STEPS = 1
    PREFETCH_FACTOR = 6 if DATA_WORKERS > 0 else None
elif PERFORMANCE_MODE == "balanced":
    DATA_WORKERS = max(3, min(6, CPU_COUNT - 1))
    TRAIN_BATCH_SIZE = 24 if GPU_AVAILABLE else 8
    GRAD_ACCUM_STEPS = 1
    PREFETCH_FACTOR = 4 if DATA_WORKERS > 0 else None
else:
    half_cpus = max(1, CPU_COUNT // 2)
    DATA_WORKERS = max(1, min(4, half_cpus))
    TRAIN_BATCH_SIZE = 16 if GPU_AVAILABLE else 8
    GRAD_ACCUM_STEPS = 1
    PREFETCH_FACTOR = 2 if DATA_WORKERS > 0 else None

HEURISTIC_BATCH_SIZE = TRAIN_BATCH_SIZE
SAFE_BATCH_CAP = None
if GPU_TOTAL_MEM_GB is not None:
    if GPU_TOTAL_MEM_GB < 12:
        SAFE_BATCH_CAP = 6
    elif GPU_TOTAL_MEM_GB < 16:
        SAFE_BATCH_CAP = 8
    elif GPU_TOTAL_MEM_GB < 20:
        SAFE_BATCH_CAP = 12

if SAFE_BATCH_CAP is not None:
    TRAIN_BATCH_SIZE = min(TRAIN_BATCH_SIZE, SAFE_BATCH_CAP)

if GPU_AVAILABLE:
    MICRO_BATCH_TARGET = 8
    TRAIN_BATCH_SIZE = min(TRAIN_BATCH_SIZE, MICRO_BATCH_TARGET)
    EFFECTIVE_TARGET_BATCH = 24
else:
    EFFECTIVE_TARGET_BATCH = TRAIN_BATCH_SIZE

GRAD_ACCUM_STEPS = max(GRAD_ACCUM_STEPS, math.ceil(EFFECTIVE_TARGET_BATCH / max(TRAIN_BATCH_SIZE, 1)))

LIGHTWEIGHT_MODEL = os.environ.get("LIFD_LIGHT_MODEL", "").strip().lower() in {"1", "true", "yes", "on"}

if GPU_TOTAL_MEM_GB is not None:
    print(f"Detected GPU memory: {GPU_TOTAL_MEM_GB:.1f} GB")
if TRAIN_BATCH_SIZE != HEURISTIC_BATCH_SIZE:
    print(
        f"Auto-adjusted batch size from {HEURISTIC_BATCH_SIZE} to {TRAIN_BATCH_SIZE} (grad_accum={GRAD_ACCUM_STEPS})."
    )
if GRAD_ACCUM_STEPS > 1:
    effective_batch = TRAIN_BATCH_SIZE * GRAD_ACCUM_STEPS
    print(f"Effective batch size via accumulation: ~{effective_batch}")
if LIGHTWEIGHT_MODEL:
    print("Lightweight model profile enabled (Swin backbone disabled, fused width reduced).")
else:
    print("Full backbone profile enabled (EfficientNet + Swin + UNet).")

BASE_PATH = Path(DRIVE_BASE_DIR)

from train import LossConfig, TrainConfig, run_training
from model.hybrid_forgery_detector import HybridForgeryConfig
from evaluation.eval_utils import (
    collect_visual_samples,
    evaluate_split,
    load_model_from_checkpoint,
 )

In [ ]:
from pathlib import Path
import os
from huggingface_hub import snapshot_download

DATASET_ROOT = Path("/content/data") if USE_COLAB else Path("prepared") / "CASIA2"
DATASET_ROOT.mkdir(parents=True, exist_ok=True)

HF_REPO_ID = "juhenes/lifd"
HF_REVISION = os.environ.get("HF_DATA_REVISION", "main")
HF_TOKEN = userdata.get("HUGGINGFACE_TOKEN") if userdata is not None else os.environ.get("HUGGINGFACE_TOKEN")
MANIFEST_FILENAME = "manifest.parquet"

def _locate_manifest_root(base: Path):
    candidates = [base, base / "CASIA2"]
    for candidate in candidates:
        manifest_path = candidate / MANIFEST_FILENAME
        if manifest_path.exists():
            return candidate
    return None

def _has_complete_dataset(root: Path) -> bool:
    return _locate_manifest_root(root) is not None

def ensure_dataset_ready(force_sync: bool = False) -> Path:
    if not force_sync:
        local_root = _locate_manifest_root(DATASET_ROOT)
        if local_root is not None:
            return local_root
    snapshot_dir = Path(snapshot_download(
        repo_id=HF_REPO_ID,
        revision=HF_REVISION,
        token=HF_TOKEN,
        repo_type="dataset",
    ))
    resolved_snapshot = _locate_manifest_root(snapshot_dir)
    if resolved_snapshot is None:
        raise FileNotFoundError(
            f"Downloaded dataset at {snapshot_dir} does not contain {MANIFEST_FILENAME}."
        )
    return resolved_snapshot

DATASET_ROOT = ensure_dataset_ready()

In [ ]:
from itertools import islice
from pprint import pprint
from pathlib import Path
import os

def inspect_prepared_dataset(root: Path, split: str = "train", preview: int = 5):
    root = Path(root)
    print(f"Resolved DATASET_ROOT: {root}")
    manifest_path = root / "manifest.parquet"
    print(f"Manifest present: {manifest_path.exists()} ({manifest_path})")
    if not manifest_path.exists():
        return
    try:
        import pandas as pd
        manifest_df = pd.read_parquet(manifest_path)
        print(f"Manifest rows: {len(manifest_df):,}")
        label_counts = manifest_df.groupby(["split"]).size().to_dict()
        print("Rows per split:", label_counts)
    except Exception as exc:
        print(f"Manifest read failed: {exc}")
    split_dir = root / split
    print(f"Split directory exists: {split_dir.exists()} ({split_dir})")
    if not split_dir.exists():
        return
    shards = sorted(split_dir.glob("*.tar"))
    print(f"Found {len(shards)} shard files under {split_dir}")
    for shard in islice(shards, preview):
        print(f" - {shard} | exists={shard.exists()} | size={shard.stat().st_size if shard.exists() else 'missing'}")
    missing = [str(p) for p in shards if not p.exists()]
    if missing:
        print("Missing shards:", missing[:5], "...")

inspect_prepared_dataset(Path(DATASET_ROOT))

In [ ]:
default_train_cfg = TrainConfig()
default_model_cfg = HybridForgeryConfig()
print("Default TrainConfig:")
pprint(asdict(default_train_cfg))
print("Default HybridForgeryConfig:")
pprint(asdict(default_model_cfg))

In [ ]:
loss_config = LossConfig(
    use_bce=True,
    bce_weight=1.0,
    use_dice=True,
    dice_weight=1.0,
    use_focal_tversky=True,
    focal_tversky_weight=0.5,
    focal_alpha=0.7,
    focal_beta=0.3,
    focal_gamma=1.2,
    use_boundary=True,
    boundary_weight=0.15,
    apply_lovasz=True,
    lovasz_weight=0.2,
)
loss_config

In [ ]:
train_config = TrainConfig(
    prepared_root=str(DATASET_ROOT),
    train_split="train",
    val_split="val",
    target_size=384,
    batch_size=TRAIN_BATCH_SIZE,
    num_epochs=30,
    learning_rate=1e-4,
    weight_decay=1e-2,
    num_workers=DATA_WORKERS,
    prefetch_factor=PREFETCH_FACTOR,
    persistent_workers=DATA_WORKERS > 0,
    pin_memory=True,
    grad_accumulation_steps=GRAD_ACCUM_STEPS,
    grad_clip_norm=1.0,
    log_interval=10,
    checkpoint_dir=str(BASE_PATH / "checkpoints"),
    checkpoint_interval=1,
    save_best_only=False,
    use_amp=True,
    resume_from=str(BASE_PATH / "checkpoints" / "epoch_001.pt"),
    loss_config=loss_config,
    balance_real_fake=True,
    balanced_positive_ratio=0.6,
    eval_thresholds=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    primary_eval_threshold=0.5,
    early_stopping_patience=6,
    early_stopping_min_delta=5e-4,
)

In [ ]:
model_config_kwargs = dict(
    use_efficientnet=True,
    use_swin=not LIGHTWEIGHT_MODEL,
    use_segformer=False,
    use_unet_decoder=True,
    use_skip_connections=True,
    pretrained_backbones=True,
    fused_channels=192 if LIGHTWEIGHT_MODEL else 256,
    use_noise_branch=True,
    noise_branch_base_channels=32 if LIGHTWEIGHT_MODEL else 48,
    noise_branch_num_stages=4,
    noise_branch_use_residual=True,
    noise_branch_use_high_pass=True,
    backbone_input_size=train_config.target_size,
    gradient_checkpointing=not LIGHTWEIGHT_MODEL,
)

train_config.model_config = HybridForgeryConfig(**model_config_kwargs)

In [ ]:
print("Resolved device:", train_config.resolved_device())
print("TrainConfig overrides:")
pprint(asdict(train_config))
print("HybridForgeryConfig overrides:")
pprint(asdict(train_config.model_config))

### Optional Dry Run
Flip the flag in the next cell to execute a 1-epoch, few-batch sanity check (uses `max_train_batches` / `max_val_batches`) before kicking off the full training run.

In [ ]:
ENABLE_DRY_RUN = False
if ENABLE_DRY_RUN:
    dry_run_config = deepcopy(train_config)
    dry_run_config.num_epochs = 1
    dry_run_config.batch_size = min(2, train_config.batch_size)
    dry_run_config.max_train_batches = 1
    dry_run_config.max_val_batches = 1
    dry_run_config.checkpoint_dir = str(Path(train_config.checkpoint_dir) / "dry_run")
    print("Dry run settings:", {
        "num_epochs": dry_run_config.num_epochs,
        "batch_size": dry_run_config.batch_size,
        "max_train_batches": dry_run_config.max_train_batches,
        "max_val_batches": dry_run_config.max_val_batches,
        "checkpoint_dir": dry_run_config.checkpoint_dir,
    })
    dry_run_history = run_training(dry_run_config)
else:
    print("Dry run skipped. Set ENABLE_DRY_RUN = True to execute the smoke test.")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
history = run_training(train_config)
history